# MLHW 6 Kernel K-means and Spectral Clustering
**312657018 統計碩二 陳佳妘**

### Data
Two 100*100 images are provided (image1.png & image2.png), and each pixel in the image should be treated as a data point, which means there are 10000 data points in each image.

### Kernel
For both kernel k-means and spectral clustering, please use the new kernel defined below
to compute the Gram matrix.
$$k(x, x') = e^{-\gamma_s∥𝑆(𝑥)−𝑆(𝑥^′)∥^2} × 𝑒^{−\gamma_c∥𝐶(𝑥)−𝐶(𝑥^′)∥^2}$$
This new defined kernel is basically multiplying two RBF kernels in order to consider
spatial similarity and color similarity at the same time. S(𝑥) is the spatial information (i.e.
the coordinate of the pixel) of data 𝑥, and C(𝑥) is the color information (i.e. the RGB
values) of data 𝑥. Both $\gamma_s$ and $\gamma_c$ are hyper-parameters which you can tune in your own 𝑠𝑐
way.

## Import & Read File

In [9]:
import cv2
import numpy as np
from scipy.spatial.distance import cdist
from PIL import Image, ImageColor
import matplotlib.pyplot as plt
import imageio
import os

In [10]:
try:
    os.mkdir("./k_means_final")
    os.mkdir("./k_means_gif")
    os.mkdir("./spectral_final")
    os.mkdir("./spectral_gif")
    os.mkdir("./e_space_2dim")
    os.mkdir("./e_space_3dim")
except OSError:
    print("dir already exist")

dir already exist


In [11]:
def read_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    C = img.reshape(-1, 3)
    S = np.array([(i, j) for i in range(img.shape[0]) for j in range(img.shape[1])])
    return img.shape[:2], C, S

img1_size, C1, S1 = read_img('image1.png')
img2_size, C2, S2 = read_img('image2.png')

## Utils

In [12]:
class Utils:
    @staticmethod
    def kernel(gamma_S, S, gamma_C, C):
        result = np.exp(-gamma_S * cdist(S, S, 'sqeuclidean')) * \
            np.exp(-gamma_C * cdist(C, C, 'sqeuclidean'))
        return result
    
    @staticmethod
    def compute_laplacian(Gram, mode_s, filename):
        """計算拉普拉斯矩陣"""
        if os.path.exists(f"compute_npy/Laplacian_s{mode_s}_" + filename + ".npy"):
            print('Loading L...')
            L = np.load(f"compute_npy/Laplacian_s{mode_s}_" + filename + ".npy")

        else:
            print('Computing L...')
            W = Gram
            D = np.diag(np.sum(W, axis=1))
            L = D - W
            if mode_s == 0:
                D_sqrt_inv = np.diag(1 / np.sqrt(np.diag(D)))
                L = D_sqrt_inv @ L @ D_sqrt_inv
            np.save(f"compute_npy/Laplacian_s{mode_s}_" + filename + ".npy", L)
        return L
    
    @staticmethod
    def compute_eigen(L, mode_s, filename):
        if (os.path.exists(f"compute_npy/eval_s{mode_s}_" + filename+".npy") and
            os.path.exists(f"compute_npy/evec_s{mode_s}_" + filename+".npy")):
            print('Loading eigenValue and eigenVector...')
            e_val = np.load(f"compute_npy/eval_s{mode_s}_" + filename+".npy")
            e_vec = np.load(f"compute_npy/evec_s{mode_s}_" + filename+".npy")
        else:
            print('Computing eigenValue and eigenVector...')
            e_val, e_vec = np.linalg.eig(L)
            np.save(f"compute_npy/eval_s{mode_s}_" + filename+".npy", e_val)
            np.save(f"compute_npy/evec_s{mode_s}_" + filename+".npy", e_vec)
        return e_val, e_vec

## Clustering

In [13]:
class Kmeans_clustering(Utils):
    def __init__(self, k, mode, mode_s=None):
        self.k = k
        self.mode = mode
        self.mode_s = mode_s
    
    def initialize(self, Gram):
        mean = np.zeros((self.k, Gram.shape[1]), dtype=Gram.dtype)
        
        # random initialize
        if self.mode == 0:    
            centers = np.random.randint(0, Gram.shape[0], self.k)
            mean = Gram[centers, :]
        
        # k-means++
        elif self.mode == 1:
            centers = [Gram[np.random.choice(Gram.shape[0])]]
            for _ in range(1, self.k):
                dist = np.min([np.linalg.norm(Gram - c, axis=1)**2 for c in centers], axis=0)
                probs = dist / np.sum(dist)
                update_center = Gram[np.random.choice(Gram.shape[0], p=probs)]
                centers.append(update_center)
            mean = np.array(centers)
            
        return mean
            
    def kmeans(self, Gram):
        history = []
        # initialize centers
        update_mean = self.initialize(Gram)
        mean = np.zeros(update_mean.shape, dtype=Gram.dtype)
        
        while np.linalg.norm(update_mean - mean) > 1e-8:
            # E-step: classify all samples according to closet μk
            clusters = np.zeros(Gram.shape[0], dtype=int)
            for n in range(Gram.shape[0]):
                J = []
                for k in range(self.k):
                    J.append(np.linalg.norm(Gram[n] - update_mean[k]))
                clusters[n] = np.argmin(J)
            history.append(clusters)
            
            # M-step: re-compute the mean
            mean = update_mean
            update_mean = np.zeros(update_mean.shape, dtype=Gram.dtype)
            cluster_size = np.zeros(self.k)
            for n in range(Gram.shape[0]):
                update_mean[clusters[n]] += Gram[n]
                cluster_size[clusters[n]] += 1
            
            for k in range(self.k):
                if cluster_size[k] > 0:
                    update_mean[k] /= cluster_size[k]
                else: 
                    update_mean[k] = mean[k]
            
            print(f'Iteration: {len(history)}')
        return history
    
    def spectral(self, Gram, filename):
        L = self.compute_laplacian(Gram, self.mode_s, filename)
        e_val, e_vec = self.compute_eigen(L, self.mode_s, filename)
        
        e_val_sort_idx = np.argsort(e_val)
        U = e_vec[:, e_val_sort_idx[1: self.k+1]]
        
        # Normalized
        if self.mode_s == 0:
            U /= np.linalg.norm(U, axis=1, keepdims=True)
        return U

## Plot

In [14]:
class PlotResult(Kmeans_clustering):
    def __init__(self, k, mode, mode_s=None):
        super().__init__(k, mode, mode_s)
        
    def plot_gif(self, Gram, image_size, filename, clustering_type):
        history = self.kmeans(Gram) 
        iteration = len(history)

        gif = []
        color = [ImageColor.getrgb('Salmon'), ImageColor.getrgb('Gold'), 
                 ImageColor.getrgb('Forestgreen'), ImageColor.getrgb('Darkviolet')]
        for i in range(iteration):
            gif.append(Image.new('RGB', image_size))
            for y in range(image_size[0]):
                for x in range(image_size[1]):
                    gif[i].putpixel((x, y), color[history[i][y * image_size[0] + x]])
        
        if clustering_type == 'kmeans':
            gif[0].save(f"./k_means_gif/{filename}_mode{self.mode}_k{self.k}.gif",
                        format='GIF',
                        save_all=True,
                        append_images=gif[1:],
                        duration=400, loop=0)
            gif[-1].save(f"./k_means_jpg/{filename}_mode{self.mode}_k{self.k}.jpg", format='JPEG')
            print(f'-------- Save for:{filename}, k:{self.k}, mode:{self.mode} --------')
            
        elif clustering_type == 'spectral':
            gif[0].save(f"./spectral_gif/{filename}_mode{self.mode}_k{self.k}_s{self.mode_s}.gif",
                        format='GIF',
                        save_all=True,
                        append_images=gif[1:],
                        duration=400, loop=0)
            gif[-1].save(f"./spectral_jpg/{filename}_mode{self.mode}_k{self.k}_s{self.mode_s}.jpg", format='JPEG')
            print(f'-------- Save for:{filename}, k:{self.k}, mode:{self.mode} s:{self.mode_s}--------')
        
    def plot2D(self, Gram, filename):
        history = self.kmeans(Gram) 
        U = self.spectral(Gram, filename)
        # print(U.shape)
        
        x = U[:, 0]
        y = U[:, 1]
        plt.xlabel("1st dim")
        plt.ylabel("2nd dim")
        for i in range(self.k):
            plt.scatter(x[history[-1]==i], y[history[-1]==i], marker='o')
        plt.title("coordinates in the eigenspace of graph Laplacian")
        # plt.show()
        plt.savefig(f'spectral_2dim/{filename}_s{self.mode_s}.jpg', format='JPEG')
        plt.close()

    def plot3D(self, Gram, filename):
        history = self.kmeans(Gram) 
        U = self.spectral(Gram, filename)
        U = np.real(U)
        # print(U.shape)
        
        fig = plt.figure()
        ax = fig.add_subplot(111, projection="3d")
        x = U[:, 0]
        y = U[:, 1]
        z = U[:, 2]
        ax.set_xlabel("1st dim")
        ax.set_ylabel("2nd dim")
        ax.set_zlabel("3rd dim")
        plt.title("coordinates in the eigenspace of graph Laplacian")
        for i in range(self.k):
            ax.scatter(x[history[-1]==i], y[history[-1]==i], z[history[-1]==i], '.')
        # plt.show()
        plt.savefig(f'spectral_3dim/{filename}_s{self.mode_s}.jpg', format='JPEG')
        plt.close()

In [16]:
def plot_for_all_combinations(gram, img_size, modes, ks, filename, clustering_type, modes_s=None):
    """
    統一處理 K-means 和 Spectral 分群的結果
    """
    for mode in modes:
        for k in ks:
            if clustering_type == 'spectral':
                for s in modes_s:
                    # 創建對應的 kernel_clustering 實例
                    spec = PlotResult(k, mode, s)
                    U = spec.spectral(gram, filename)
                    spec.plot_gif(U, img_size, filename, clustering_type)
                    if k == 2:
                        spec.plot2D(U, filename)
                    elif k == 3:
                        spec.plot3D(U, filename)
            else:
                kmeans = PlotResult(k, mode)    
                kmeans.plot_gif(gram, img_size, filename, clustering_type)
                
# setting parameters
gram1 = Utils.kernel(1e-3, S1, 1e-3, C1)  
gram2 = Utils.kernel(1e-3, S2, 1e-3, C2)  
modes = [0, 1]  # 0: kmeans, 1: kmeans++
ks = [2, 3, 4] 
modes_s = [0, 1] # 0: normalized, 1: unnormalized


In [10]:
plot_for_all_combinations(gram1, img1_size, modes, ks, 'image1', 'kmeans')
plot_for_all_combinations(gram2, img2_size, modes, ks, 'image2', 'kmeans')

Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
-------- Save for:image1, k:2, mode:0 --------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration: 21
Iteration: 22
Iteration: 23
Iteration: 24
Iteration: 25
Iteration: 26
Iteration: 27
Iteration: 28
Iteration: 29
Iteration: 30
Iteration: 31
Iteration: 32
Iteration: 33
Iteration: 34
Iteration: 35
Iteration: 36
Iteration: 37
-------- Save for:image1, k:3, mode:0 --------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
Iteration: 20
Iteration

In [17]:
plot_for_all_combinations(gram1, img1_size, modes, ks, 'image1', 'spectral', modes_s)
plot_for_all_combinations(gram2, img2_size, modes, ks, 'image2', 'spectral', modes_s)

Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
-------- Save for:image1, k:2, mode:0 s:0--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Loading L...
Loading eigenValue and eigenVector...


/Users/jolene/anaconda3/envs/MLHW/lib/python3.12/site-packages/matplotlib/cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/Users/jolene/anaconda3/envs/MLHW/lib/python3.12/site-packages/matplotlib/collections.py:197: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)


Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
-------- Save for:image1, k:2, mode:0 s:1--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
-------- Save for:image1, k:3, mode:0 s:0--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
-------- Save for:image1, k:3, mode:0 s:1--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iter

/Users/jolene/anaconda3/envs/MLHW/lib/python3.12/site-packages/matplotlib/cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/Users/jolene/anaconda3/envs/MLHW/lib/python3.12/site-packages/matplotlib/collections.py:197: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)


Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
-------- Save for:image1, k:2, mode:1 s:1--------
Iteration: 1
Iteration: 2
Iteration: 3
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
-------- Save for:image1, k:3, mode:1 s:0--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
-------- Save for:image1, k:3, mode:1 s:1--------
Iteration: 1
Iteration: 2
Iteration: 3
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Itera

/Users/jolene/anaconda3/envs/MLHW/lib/python3.12/site-packages/matplotlib/cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/Users/jolene/anaconda3/envs/MLHW/lib/python3.12/site-packages/matplotlib/collections.py:197: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)


Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
-------- Save for:image2, k:2, mode:0 s:1--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
-------- Save for:image2, k:3, mode:0 s:0--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iter

/Users/jolene/anaconda3/envs/MLHW/lib/python3.12/site-packages/matplotlib/cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
/Users/jolene/anaconda3/envs/MLHW/lib/python3.12/site-packages/matplotlib/collections.py:197: ComplexWarning: Casting complex values to real discards the imaginary part
  offsets = np.asanyarray(offsets, float)


Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
-------- Save for:image2, k:2, mode:1 s:1--------
Iteration: 1
Iteration: 2
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
-------- Save for:image2, k:3, mode:1 s:0--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Loading L...
Loading eigenValue and eigenVector...
Loading L...
Loading eigenValue and eigenVector...
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
-------- Save for:image2, k:3, mode:1 s:1--------
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteratio